# 인도양 쌍극자 진동지수 1년 예측 모델링

  ## 예측변수&예측인자 자료

* 예측변수
  * 인도양쌍극자진동 지수
* 예측인자
  * 해수면온도편차
  * 지상동서바람편차
  * 지상남북바람편차
  * 강수량편차
  * 전지구 이산화탄소평균
  * 전지구 메탄평균

  *주어진 자료기간은 1982년 1월부터 2023년 7월까지입니다.*

## 모델링 개요

* ConvLSTM모델을 사용한 예측
  * 온실가스를 포함한 모델과 포함하지 않는 모델의 RMSE를 비교합니다.
  * Leed Time별 RMSE를 비교하여 모델을 선택합니다.

  *예측기간은 2023년 8월부터 2024년 7월까지입니다.*

# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
# 전처리
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 모델링
import tensorflow as tf
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import Dropout
from keras.layers import Activation
from keras.backend import clear_session
from keras.callbacks import EarlyStopping

# 평가
from sklearn.metrics import *

# 시드값 고정
import random as rn
import os

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

# 경고문 무시
import warnings
warnings.filterwarnings('ignore')

## (2) 데이터 불러오기

In [ ]:
# # 구글 코랩에서 사용시 경로 코드
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/Ocean_Market

In [ ]:
# 기본예측인자
sst = xr.open_dataset('./sst.anom.mon.mean.nc', decode_times=False)['sst']
precip = xr.open_dataset('./precip.anom.mon.mean.nc', decode_times=False)['precip']
vwnd = xr.open_dataset('./vwnd.10m.anom.mon.mean.nc', decode_times=False)['vwnd']
uwnd = xr.open_dataset('./uwnd.10m.anom.mon.mean.nc', decode_times=False)['uwnd']

# 예측변수
iod = xr.open_dataset('./iod.nc', decode_times=False)['iod']

# 온실가스
co2_data = pd.read_csv('co2.csv') # 이산화탄소
ch4_data = pd.read_csv('ch4.csv') # 메탄

# 2.전처리1(기본예측인자)

## (1) 기초전처리

In [ ]:
# 시간형식 변경
times = []
for i in sst['time'].values:
    times.append(datetime(1, 1, 1) + timedelta(hours=i) - timedelta(days=2))

sst['time'] = times[:]
precip['time'] = times[:]
vwnd['time'] = times[:]
uwnd['time'] = times[:]
iod['time'] = times[:]

In [ ]:
# 서인도양 (50°E~70°E, 10°S~10°N)
minlat_w, maxlat_w = -10, 10
minlon_w, maxlon_w = 50, 70

sst_WTIO = sst.sel(lat=slice(minlat_w, maxlat_w), lon=slice(minlon_w, maxlon_w))
precip_WTIO = precip.sel(lat=slice(minlat_w, maxlat_w), lon=slice(minlon_w, maxlon_w))
vwnd_WTIO = vwnd.sel(lat=slice(minlat_w, maxlat_w), lon=slice(minlon_w, maxlon_w))
uwnd_WTIO = uwnd.sel(lat=slice(minlat_w, maxlat_w), lon=slice(minlon_w, maxlon_w))

# 동인도양 (90°E~110°E, 10°S~10°N)
minlat_e, maxlat_e = -10, 10
minlon_e, maxlon_e = 90, 110

sst_SETIO = sst.sel(lat=slice(minlat_e, maxlat_e), lon=slice(minlon_e, maxlon_e))
precip_SETIO = precip.sel(lat=slice(minlat_e, maxlat_e), lon=slice(minlon_e, maxlon_e))
vwnd_SETIO = vwnd.sel(lat=slice(minlat_e, maxlat_e), lon=slice(minlon_e, maxlon_e))
uwnd_SETIO = uwnd.sel(lat=slice(minlat_e, maxlat_e), lon=slice(minlon_e, maxlon_e))

In [ ]:
# lev 변수 삭제 (lev값이 1개만 존재하여 유의미하지 않다)
vwnd_WTIO = vwnd_WTIO.sel(lev=10)
vwnd_SETIO = vwnd_SETIO.sel(lev=10)

uwnd_WTIO = uwnd_WTIO.sel(lev=10)
uwnd_SETIO = uwnd_SETIO.sel(lev=10)

In [ ]:
# WTIO, SETIO 데이터 결합
# 두 데이터의 lat의 범위가 같으므로 lat을 기준으로 합친다 # (time, lat, lon)
sst_data = np.concatenate([sst_WTIO.values, sst_SETIO.values], 2)
precip_data = np.concatenate([precip_WTIO.values, precip_SETIO.values], 2)
vwnd_data = np.concatenate([vwnd_WTIO.values, vwnd_SETIO.values], 2)
uwnd_data = np.concatenate([uwnd_WTIO.values, uwnd_SETIO.values], 2)
print('sst shape :', sst_data.shape)
print('precip shape :', precip_data.shape)
print('vwnd shape :', vwnd_data.shape)
print('uwnd shape :', uwnd_data.shape)

## (2) 결측치 처리

### 결측치 확인

In [ ]:
# nan 값 원소 개수 확인 함수
def find_nan_num(data):
    nan_values = data[np.isnan(data)] # nan 값 확인
    vnames = [name for name in globals() if globals()[name] is data] # 변수명 인쇄
    print(f'{vnames[0]} 원소 중 NaN 값 개수 : {len(nan_values)}')

In [ ]:
features_list = [sst_data, precip_data, vwnd_data, uwnd_data]
for feature in features_list:
    find_nan_num(feature)

In [ ]:
# 29개
len(sst_data[0][np.isnan(sst_data[0])])
# 지표면 때문에 시간단위 한개당 29개의 결측치가 sst_data에 생겼다.

### Cubic interpolate 방법으로 결측치 처리

In [ ]:
# Cubic interpolate를 사용한 sst_data NaN 값 채우기
result = []
for x in sst_data:
    temp = pd.DataFrame(x).interpolate(method='cubic')
    temp.iloc[19, 0] = temp.iloc[18, 0]
    result.append(temp.to_numpy())

sst_data = np.array(result)[:]

In [ ]:
find_nan_num(sst_data)

## (3) 스케일링

In [ ]:
# 3차원 MinMaxScaler
def three_d_minmax(data):
    scaler = MinMaxScaler()
    X_scale = scaler.fit_transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)

    return X_scale

In [ ]:
sst_data_scale = three_d_minmax(sst_data)
precip_data_scale = three_d_minmax(precip_data)
vwnd_data_scale = three_d_minmax(vwnd_data)
uwnd_data_scale = three_d_minmax(uwnd_data)

# 3.전처리2(온실가스)

In [ ]:
# 데이터 shape 변경 함수
def transformer(n, m, data):
    result = []
    for i in data:
        temp = [i for _ in range(n*m)]
        temp = np.array(temp)
        temp = temp.reshape(n, m)
        result.append(temp)

    return np.array(result)

In [ ]:
# CO2와 메탄은 전지구적 평균이므로 같은값으로 shape를 확장시킨다.
co2_data_re = transformer(20, 42, co2_data.to_numpy())
ch4_data_re = transformer(20, 42, ch4_data.to_numpy())

In [ ]:
# 3차원 minmax 스케일링
co2_data_scale = three_d_minmax(co2_data_re)
ch4_data_scale = three_d_minmax(ch4_data_re)

# 4.전처리3(예측변수)

In [ ]:
# lat, lon 변수 삭제
iod_data = iod.sel(lon=1, lat=1).values

In [ ]:
# iod 이상치 확인
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(iod['time'], iod_data)
plt.grid(linestyle='--', color='gray')
plt.title('Indian Ocean Dipole: Line')

plt.subplot(2, 1, 2)
sns.boxplot(x=iod_data)
plt.grid(linestyle='--', color='gray')
plt.title('Indian Ocean Dipole: Boxplot')

plt.tight_layout()
plt.show()

# 모델링 데이터셋 생성

## (1) 예측인자 reshape

In [ ]:
# shape를 맞추기 위한 MLPclass 생성
class MLP(Model):
    def __init__(self, in_chans=None, latent_dim=784, out_dim=None, act_layer=Activation('gelu'), drop_rate=None):
        super(MLP, self).__init__()
        self.fc1 = Dense(latent_dim)
        self.fc2 = Dense(out_dim)
        self.act = act_layer
        self.drop = Dropout(drop_rate)

    def call(self, x):
        x = Flatten()(x)
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)

        x = self.fc2(x)
        x = self.drop(x)

        return x

In [ ]:
# 모델 생성
def feature_reshape(data):
    clear_session()

    model = MLP(in_chans=data.shape[0]*data.shape[1]*data.shape[2], latent_dim=196, out_dim=840, drop_rate=0.2)

    output = model(data)

    temp = output.numpy()

    return temp.reshape(499, 20, 42) # sst_data.shape == (499, 20, 42)

In [ ]:
precip_data_re = feature_reshape(precip_data_scale)
vwnd_data_re = feature_reshape(vwnd_data_scale)
uwnd_data_re = feature_reshape(uwnd_data_scale)

print('sst shape :', sst_data_scale.shape)
print('New precip shape :', precip_data_re.shape)
print('New vwnd shape :', vwnd_data_re.shape)
print('New uwnd shape :', uwnd_data_re.shape)

## (2) 데이터셋 생성

In [ ]:
# 온실가스 무
feature_total = np.stack([sst_data_scale, precip_data_re, vwnd_data_re, uwnd_data_re], axis=3)

# 온실가스 유
feature_total_gas = np.stack([sst_data_scale, precip_data_re, vwnd_data_re, uwnd_data_re, co2_data_scale, ch4_data_scale], axis=3)

# 4.모델링

## (1) 함수 생성

### 시계열 데이터 분할 함수

In [ ]:
# 다변량 시퀀스를 샘플로 분할하는 함수
# input 부분만 추출 (target과 featrue가 현재 다른 데이터이므로 다른 함수를 사용)
def split_sequences_x(sequences, n_steps_in, n_steps_out=12):
    # n_steps_in : 예측에 사용하는 이전 시간단계 수
    # n_steps_out : 예측하려는 시간단계 수
    X = []
    for i in range(len(sequences)):
        # 패턴의 끝을 찾기
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # 데이터셋을 벗어났는지 확인
        if out_end_ix > len(sequences):
            break
        # 패턴의 input부분을 추출
        seq_x = sequences[i:end_ix, :]
        X.append(seq_x)
    return np.array(X)

In [ ]:
# 일변량 시퀀스를 샘플로 분할하는 함수
# output 부분만 추출 (target과 featrue가 현재 다른 데이터이므로 다른 함수를 사용)
def split_sequence_y(sequence, n_steps_in, n_steps_out=12):
    y = []
    for i in range(len(sequence)):
        # 패턴의 끝을 찾기
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # 데이터셋을 벗어났는지 확인
        if out_end_ix > len(sequence):
            break
        # 패턴의 output부분을 추출
        seq_y = sequence[end_ix:out_end_ix]
        y.append(seq_y)
    return np.array(y)

In [ ]:
def split_data(x_data, y_data, n_steps_in):
    ### 시계열 시퀸스 분할
    # target
    iod_y = split_sequence_y(y_data, n_steps_in)

    # feature
    X= split_sequences_x(x_data, n_steps_in)

    ### train, val 데이터셋 분할
    # train : val = 8 : 2
    split_num = int(X.shape[0]*0.2)
    print(f'{split_num}개의 데이터를 val로 설정')
    print('='*80)

    # features train, val 분활
    x_train, x_val = X[:-split_num], X[-split_num:]

    # target train, val 분활
    y_train, y_val = iod_y[:-split_num], iod_y[-split_num:]

    return x_train, x_val, y_train, y_val

### 모델생성 및 평가 함수

In [ ]:
def build_model(x_data, y_data, n_steps_in):
    # 데이터 준비
    x_train, x_val, y_train, y_val = split_data(x_data, y_data, n_steps_in)

    # 파라미터 설정
    verbose, epochs, batch_size = 0, 100, 32

    # 매모리 리셋
    clear_session()
    # 시드 설정
    SEED = 42
    os.environ['PYTHONHASHSEED'] = str(SEED)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    rn.seed(SEED)
    es_cb = EarlyStopping(monitor='val_loss', patience=10)

    # define model
    X_Input = Input(shape=(x_train.shape[1:]))
    X_out = ConvLSTM2D(filters=20, kernel_size=(3, 3), padding='same', return_sequences=True)(X_Input)
    X_out = BatchNormalization()(X_out)

    X_out = ConvLSTM2D(filters=20, kernel_size=(3, 3), padding='same', return_sequences=True)(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = ConvLSTM2D(filters=20, kernel_size=(3, 3), padding='same', return_sequences=True)(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=True)(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = Conv3D(filters=1, kernel_size=(1, 1, 1))(X_out)

    X_out = Flatten()(X_out)

    X_out = Dense(5040, activation='swish')(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = Dense(512, activation='swish')(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = Dense(256, activation='swish')(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = Dense(128, activation='swish')(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = Dense(64, activation='swish')(X_out)
    X_out = BatchNormalization()(X_out)

    X_out = Dropout(0.5)(X_out)

    X_out = Dense(12)(X_out)

    convlstm_model = Model(X_Input, X_out)

    convlstm_model.compile(optimizer='adam', loss='mse')

    # fit model
    convlstm_model.fit(x_train, y_train,
                    verbose=verbose, validation_split=.2, epochs=epochs,
                    batch_size=batch_size, callbacks=[es_cb], shuffle=False)

    # predict
    y_hat = convlstm_model.predict(x_val, verbose=0)

    return y_val, y_hat, convlstm_model

In [ ]:
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        rmse = mean_squared_error(actual[:, i], predicted[:, i], squared=False)
        scores.append(rmse)

    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
            score = np.sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

In [ ]:
# summarize scores
def summarize_scores(name, score, scores):
    s_scores = ', '.join([f'{s:.3f}' for s in scores])
    print(f'{name}: [{score:.3f}] {s_scores}')
    print('='*80)

    # plot scores
    months = [str(i) for i in range(1, 13)]
    plt.figure(figsize=(12, 6))
    plt.plot(months, scores, marker='o')
    plt.title(name)
    plt.xlabel('Lead Time(Months)')
    plt.ylabel('RMSE')
    plt.grid(linestyle='--', color='gray', alpha=0.7)
    plt.show()

## (2) 기존예측인자로 예측

In [ ]:
# 기존 features 2개월
n_steps_in = 2
name = 'past_2_months'

y_val, y_hat, past_2_months_model = build_model(feature_total, iod_data, n_steps_in)
past_2_months_score, past_2_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, past_2_months_score, past_2_months_scores)

In [ ]:
# 기존 features 3개월
n_steps_in = 3
name = 'past_3_months'

y_val, y_hat, past_3_months_model = build_model(feature_total, iod_data, n_steps_in)
past_3_months_score, past_3_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, past_3_months_score, past_3_months_scores)

In [ ]:
# 기존 features 6개월
n_steps_in = 6
name = 'past_6_months'

y_val, y_hat, past_6_months_model = build_model(feature_total, iod_data, n_steps_in)
past_6_months_score, past_6_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, past_6_months_score, past_6_months_scores)

In [ ]:
# 기존 features 12개월
n_steps_in = 12
name = 'past_12_months'

y_val, y_hat, past_12_months_model = build_model(feature_total, iod_data, n_steps_in)
past_12_months_score, past_12_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, past_12_months_score, past_12_months_scores)

In [ ]:
# 기존 features 24개월
n_steps_in = 24
name = 'past_24_months'

y_val, y_hat, past_24_months_model = build_model(feature_total, iod_data, n_steps_in)
past_24_months_score, past_24_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, past_24_months_score, past_24_months_scores)

## (3) 온실가스를 포함한 예측

In [ ]:
# new features 2개월
n_steps_in = 2
name = 'gas_past_2_months'

y_val, y_hat, gas_past_2_months_model = build_model(feature_total_gas, iod_data, n_steps_in)
gas_past_2_months_score, gas_past_2_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, gas_past_2_months_score, gas_past_2_months_scores)

In [ ]:
# new features 3개월
n_steps_in = 3
name = 'gas_past_3_months'

y_val, y_hat, gas_past_3_months_model = build_model(feature_total_gas, iod_data, n_steps_in)
gas_past_3_months_score, gas_past_3_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, gas_past_3_months_score, gas_past_3_months_scores)

In [ ]:
# new features 6개월
n_steps_in = 6
name = 'gas_past_6_months'

y_val, y_hat, gas_past_6_months_model = build_model(feature_total_gas, iod_data, n_steps_in)
gas_past_6_months_score, gas_past_6_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, gas_past_6_months_score, gas_past_6_months_scores)

In [ ]:
# new features 12개월
n_steps_in = 12
name = 'gas_past_12_months'

y_val, y_hat, gas_past_12_months_model = build_model(feature_total_gas, iod_data, n_steps_in)
gas_past_12_months_score, gas_past_12_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, gas_past_12_months_score, gas_past_12_months_scores)

In [ ]:
# new features 24개월
n_steps_in = 24
name = 'gas_past_24_months'

y_val, y_hat, gas_past_24_months_model = build_model(feature_total_gas, iod_data, n_steps_in)
gas_past_24_months_score, gas_past_24_months_scores = evaluate_forecasts(y_val, y_hat)
summarize_scores(name, gas_past_24_months_score, gas_past_24_months_scores)

# 5. 결과

## (1) 각 모델별 RMSE 비교

In [ ]:
months = [str(i) for i in range(1, 13)]
plt.figure(figsize=(12, 6))
plt.plot(months, past_2_months_scores, marker='o', label='org_2')
plt.plot(months, past_3_months_scores, marker='o', label='org_3')
plt.plot(months, past_6_months_scores, marker='o', label='org_6')
plt.plot(months, past_12_months_scores, marker='o', label='org_12')
plt.plot(months, past_24_months_scores, marker='o', label='org_24')

plt.plot(months, gas_past_2_months_scores, marker='o', label='gas_2')
plt.plot(months, gas_past_3_months_scores, marker='o', label='gas_3')
plt.plot(months, gas_past_6_months_scores, marker='o', label='gas_6')
plt.plot(months, gas_past_12_months_scores, marker='o', label='gas_12')
plt.plot(months, gas_past_24_months_scores, marker='o', label='gas_24')
plt.ylim(0.4, 1) # y축 범위 지정(0.4부터 1까지의 값만 표시)
plt.title('Result')
plt.xlabel('Lead Time(Months)')
plt.ylabel('RMSE')
plt.grid(linestyle='--', color='gray', alpha=0.7)
plt.legend()
plt.show()

## (2) 12개월 IOD 예측

In [ ]:
IOD_pred = [0 for _ in range(12)]

In [ ]:
# Leed Time 1, 3, 6 (past_3_months_model)
n = 3
x_test_3 = feature_total[-n:]
x_test_3 = x_test_3.reshape(1, n, feature_total.shape[1], feature_total.shape[2], feature_total.shape[3])
past_3_pred = past_3_months_model.predict(x_test_3)

# 데이터 적용
leed_time = [1, 3, 6]
for i in leed_time:
    IOD_pred[i-1] = past_3_pred[0][i-1]

In [ ]:
# Leed Time 5, 7, 10, 11 (gas_past_2_months_model)
n = 2
x_test_2_gas = feature_total_gas[-n:]
x_test_2_gas = x_test_2_gas.reshape(1, n, feature_total_gas.shape[1], feature_total_gas.shape[2], feature_total_gas.shape[3])
past_2_pred_gas = gas_past_2_months_model.predict(x_test_2_gas)

# 데이터 적용
leed_time = [5, 7, 10, 11]
for i in leed_time:
    IOD_pred[i-1] = past_2_pred_gas[0][i-1]

In [ ]:
# Leed Time 2, 4, 9, 12 (gas_past_3_months_model)
n = 3
x_test_3_gas = feature_total_gas[-n:]
x_test_3_gas = x_test_3_gas.reshape(1, n, feature_total_gas.shape[1], feature_total_gas.shape[2], feature_total_gas.shape[3])
past_3_pred_gas = gas_past_3_months_model.predict(x_test_3_gas)

# 데이터 적용
leed_time = [2, 4, 9, 12]
for i in leed_time:
    IOD_pred[i-1] = past_3_pred_gas[0][i-1]

In [ ]:
# Leed Time 8 (gas_past_6_months_model)
n = 6
x_test_6_gas = feature_total_gas[-n:]
x_test_6_gas = x_test_6_gas.reshape(1, n, feature_total_gas.shape[1], feature_total_gas.shape[2], feature_total_gas.shape[3])
past_6_pred_gas = gas_past_6_months_model.predict(x_test_6_gas)

# 데이터 적용
leed_time = [8]
for i in leed_time:
    IOD_pred[i-1] = past_6_pred_gas[0][i-1]

In [ ]:
print('IOD 12months predict : ', IOD_pred)

## (3) 12개월 IOD 예측결과

In [ ]:
print('IOD 12months predict : ', IOD_pred)

||2023/08/01|2023/09/01|2023/10/01|2023/11/01|2023/12/01|2024/01/01|2024/02/01|2024/03/01|2024/04/01|2024/05/01|2024/06/01|2024/07/01|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|IOD|0.11150973|0.09338133|0.054903753|0.109347954|0.10077257|0.0029698047|0.045509547|0.08101446|0.15675062|0.10012395|-0.0009515595|-0.06319296|

